In [1]:
from dataset.sequence import SkeletonClusteredSequence, ClusteredSequence
import pickle as pkl
import mxnet as mx
from tqdm import tqdm
from copy import copy

with open('clustered_sequences.pkl', 'rb') as file:
    sequences = pkl.load(file)

with open('skeleton_sequences.pkl', 'rb') as file:
    skeletons = pkl.load(file)



In [2]:
clustered_skeleton_sequences = []
for sk_seq in skeletons:
    for seq in sequences:
        if sk_seq.name_pattern == seq.name_pattern and sk_seq.start_frame == seq.gt['frame'].min():
            clustered_skeleton_sequences.append(SkeletonClusteredSequence(seq, sk_seq))
            break
len(clustered_skeleton_sequences)

1692

In [3]:
from utils.missing_data import remove_ellipsis
import numpy as np


class SinglePersonSequenceHandler:
    def __init__(self, seq: SkeletonClusteredSequence):
        self.seq = seq

    def get_person_num(self):
        return len(self.seq.person_features)

    def get_person_ids(self):
        return list(self.seq.person_features.keys())

    def get_single_person_attributes(self, person_id):
        array_index = self.get_person_ids().index(person_id)
        klass = self.seq.cluster_labels[array_index]
        skeleton_seq, confidence_seq, classes_seq, bboxes_seq, scores_seq = self._get_skeleton_seq(person_id)
        return skeleton_seq, confidence_seq, classes_seq, bboxes_seq, scores_seq, klass

    def get_single_person_as_train_record(self, person_id):
        array_index = self.get_person_ids().index(person_id)
        klass = self.seq.cluster_labels[array_index]
        skeleton_seq = []
        for skeleton_iter, person_ids in zip(self.seq.skeleton_seq.skeletons_seq, self.seq.skeleton_seq.person_ids):
            if person_id not in person_ids:
                skeleton_seq.append(...)
                continue
            person_idx = np.where(person_ids == float(person_id))[0]
            skeleton_seq.append(skeleton_iter[person_idx])
        return remove_ellipsis(skeleton_seq), klass

    def _get_skeleton_seq(self, person_id):
        skeleton_seq = []
        confidence_seq = []
        classes_seq = []
        bboxes_seq = []
        scores_seq = []
        for skeleton_iter, confidence_iter, classes_iter, bboxes_iter, scores_iter, person_ids in zip(
                self.seq.skeleton_seq.skeletons_seq,
                self.seq.skeleton_seq.confidence_seq,
                self.seq.skeleton_seq.classes_seq,
                self.seq.skeleton_seq.bboxes_seq,
                self.seq.skeleton_seq.scores_seq,
                self.seq.skeleton_seq.person_ids):
            if person_id not in person_ids:
                skeleton_seq.append(...)
                confidence_seq.append(...)
                classes_seq.append(...)
                bboxes_seq.append(...)
                scores_seq.append(...)
                continue
            person_idx = np.where(person_ids == float(person_id))[0]
            skeleton_seq.append(skeleton_iter[person_idx])
            confidence_seq.append(confidence_iter[person_idx])
            classes_seq.append(classes_iter[0][person_idx])
            bboxes_seq.append(bboxes_iter[0][person_idx])
            scores_seq.append(scores_iter[0][person_idx])
        return remove_ellipsis(skeleton_seq), remove_ellipsis(confidence_seq), remove_ellipsis(
            classes_seq), remove_ellipsis(bboxes_seq), remove_ellipsis(scores_seq)


handler = SinglePersonSequenceHandler(clustered_skeleton_sequences[0])
skeleton, confidence, classes, bboxes, scores, single_klass = handler.get_single_person_attributes(4)


In [4]:
import tensorflow as tf
import keras

with open('single_X.pkl', 'rb') as file:
    X = np.array(pkl.load(file))

with open('single_y.pkl', 'rb') as file:
    y = np.array(pkl.load(file))

X = X.reshape((X.shape[0], X.shape[1], X.shape[2] * X.shape[3]))
print(X.shape)
print(y.shape)
hidden_state_ext = keras.models.load_model('hidden_state_ext.keras')
# print(X[0][None,].shape)
w1 = hidden_state_ext.predict(X[0][None,])
w2 = hidden_state_ext.predict(X[1][None,])
timesteps, hidden_shape = hidden_state_ext.layers[-1].output_shape[1:]
timesteps, hidden_shape


2023-01-08 16:26:10.471036: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 16:26:11.201323: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rafa/anaconda3/envs/sportstrack/lib/python3.7/site-packages/cv2/../../lib64:
2023-01-08 16:26:11.201375: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rafa/anaconda3/envs/sportstrack/lib/pytho

(19632, 32, 34)
(19632,)


2023-01-08 16:26:11.808539: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-08 16:26:11.809505: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-08 16:26:11.809586: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-08 16:26:11.809871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 2s 2s/step


2023-01-08 16:26:15.844612: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8600


1/1 [==============================] - 0s 20ms/step


(32, 256)

In [5]:
from keras.models import Sequential

pooler = Sequential()
max_people_numb = 20
pooler.add(tf.keras.layers.MaxPooling2D(pool_size=(1, max_people_numb), strides=(1, 1), padding='valid'))
pooler.add(tf.keras.layers.Reshape((timesteps, hidden_shape)))
# pooler.add(tf.keras.layers.Reshape((1,hidden_shape)))
pooler.compile('adam', 'mean_squared_error')
# print(w1)
# print(w2)

# inp =(np.array([w1.reshape(timesteps * hidden_shape,1), w2.reshape(timesteps * hidden_shape,1)]))
# inp=(inp.T.reshape(1,timesteps * hidden_shape,max_people_numb , 1))
#
# w3 = pooler.predict(inp)
# print(w3.shape)


In [6]:
from tqdm import tqdm
import gluoncv
import cv2
from utils.normalizers import skeleton_normalizer

X = []
y = []
for seq in tqdm(clustered_skeleton_sequences):
    handler = SinglePersonSequenceHandler(seq)
    sequence_hidden_states = []
    for person_id in handler.get_person_ids():
        skeleton, single_klass = handler.get_single_person_as_train_record(person_id)
        normalized_skeletons = np.squeeze(
            np.array([np.reshape(skeleton_normalizer(single_skeleton), (1, 34)) for single_skeleton in skeleton]))
        hidden_state = hidden_state_ext.predict(normalized_skeletons[None, :], verbose=0)
        sequence_hidden_states.append(hidden_state)
    first_20_people = sequence_hidden_states[:max_people_numb]
    first_20_people += [[np.array([[-99999999] * hidden_shape]*timesteps)[None, :]] * (max_people_numb - len(first_20_people))][0]
    shaped = (np.array([w.reshape(timesteps * hidden_shape, 1) for w in first_20_people]))
    inp = (shaped.T.reshape(1, timesteps * hidden_shape, max_people_numb, 1))
    pooled_over_people = pooler.predict(inp, verbose=0)
    X.append(pooled_over_people)
    y.append(seq.klass)

    # first_100_people = hidden_states[:100]
    # X.append(np.squeeze(np.array(normalized_skeletons)))
    # y.append(single_klass)
print(len(X), len(y))
X[0].shape, y[0]

/home/rafa/anaconda3/envs/sportstrack/lib/python3.7/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.1` and `torch==1.8.1+cu111` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '
  0%|          | 0/1692 [00:00<?, ?it/s]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


100%|██████████| 1692/1692 [16:46<00:00,  1.68it/s]

1692 1692


((1, 32, 256), 'football')

In [8]:
with open('multi_X.pkl', 'wb') as file:
    pkl.dump(X, file)

with open('multi_y.pkl', 'wb') as file:
    pkl.dump(y, file)